# 꽃 분류를 위한 이미지 데이터 전처리

## 개요
이 노트북은 꽃 분류 모델 학습을 위한 이미지 데이터 전처리를 수행합니다.

### 주요 기능
- 이미지 파일들을 읽어서 NumPy 배열로 변환
- 일정한 크기로 리사이즈하여 학습용 데이터 생성
- NPZ 형식으로 저장하여 모델 학습에 사용

### 데이터 구조
- **입력**: 폴더별로 분류된 꽃 이미지들 (daisy, dandelion 등)
- **출력**: 80x80x3 크기로 통일된 NumPy 배열 데이터

### 참고사항
- `imghdr`는 Python 3.11+에서 deprecated, 3.13에서 제거됨
- 대안으로 PIL의 이미지 형식 확인 기능 사용


In [4]:
# 필요한 라이브러리 설치 (필요시 실행)

import subprocess
import sys

def install_package(package):
    """패키지 설치 함수"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} 설치 완료!")
    except Exception as e:
        print(f"❌ {package} 설치 실패: {e}")

# 필요한 패키지 목록
required_packages = [
    "numpy",
    "pillow",  # PIL
    "pandas"
]

print("📦 필요한 패키지 설치 확인...")
for package in required_packages:
    try:
        __import__(package.replace("-", "_"))
        print(f"✅ {package} 이미 설치됨")
    except ImportError:
        print(f"⚠️ {package} 설치 중...")
        install_package(package)

print("\n🎉 패키지 설치 확인 완료!")


📦 필요한 패키지 설치 확인...
✅ numpy 이미 설치됨
⚠️ pillow 설치 중...
✅ pillow 설치 완료!
✅ pandas 이미 설치됨

🎉 패키지 설치 확인 완료!


In [5]:
# 필요한 라이브러리 import (imghdr 없이)
import numpy as np 
import os 
import random 
from PIL import Image
import pandas as pd

# imghdr 대신 사용할 이미지 형식 확인 함수
def check_image_format(file_path):
    """
    PIL을 사용하여 이미지 형식을 확인하는 함수
    imghdr의 대안으로 사용
    """
    try:
        with Image.open(file_path) as img:
            return img.format.lower() if img.format else None
    except Exception:
        return None

print("✅ 라이브러리 import 완료!")
print("🔧 이미지 형식 확인 함수 정의 완료!")


✅ 라이브러리 import 완료!
🔧 이미지 형식 확인 함수 정의 완료!


In [6]:
# 데이터 경로 설정
base_path = "C:/Users/ryan9/문서/GitHub/SeSac-AI-Developer-Notes-2025/07_Deep_Learning/data/flowers"

print(f"📂 데이터 경로: {base_path}")

# 경로 존재 여부 확인
if os.path.exists(base_path):
    print("✅ 데이터 경로가 존재합니다.")
    print("📁 하위 폴더 목록:")
    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path):
            print(f"  - {folder}")
            # 각 폴더 내 파일 개수 확인
            try:
                subfolders = os.listdir(folder_path)
                for subfolder in subfolders:
                    subfolder_path = os.path.join(folder_path, subfolder)
                    if os.path.isdir(subfolder_path):
                        file_count = len([f for f in os.listdir(subfolder_path) 
                                        if os.path.isfile(os.path.join(subfolder_path, f))])
                        print(f"    └── {subfolder}: {file_count}개 파일")
            except Exception as e:
                print(f"    └── 파일 개수 확인 실패: {e}")
else:
    print("❌ 데이터 경로가 존재하지 않습니다.")


📂 데이터 경로: C:/Users/ryan9/문서/GitHub/SeSac-AI-Developer-Notes-2025/07_Deep_Learning/data/flowers
✅ 데이터 경로가 존재합니다.
📁 하위 폴더 목록:
  - test
    └── daisy: 77개 파일
    └── dandelion: 105개 파일
  - train
    └── daisy: 529개 파일
    └── dandelion: 746개 파일
  - valid
    └── daisy: 163개 파일
    └── dandelion: 201개 파일


In [7]:
def makeData(flower_name, label, isTrain=True):
    """
    이미지 데이터를 처리하여 NPZ 파일로 저장하는 함수
    
    Args:
        flower_name (str): 꽃 종류 이름 (예: "daisy", "dandelion")
        label (int): 레이블 번호 (0: daisy, 1: dandelion 등)
        isTrain (bool): True면 train 데이터, False면 test 데이터
    """
    
    # 학습용/테스트용 데이터 경로 설정
    if isTrain:
        path = base_path + "/train/" + flower_name 
    else:
        path = base_path + "/test/" + flower_name 
    
    print(f"📂 처리할 경로: {path}")
    
    # 데이터와 레이블을 저장할 리스트 초기화
    data = []
    labels = [] 
    
    # 폴더 내 모든 파일에 대해 처리
    file_count = 0
    error_count = 0
    
    for filename in os.listdir(path):
        try:
            file_path = os.path.join(path, filename)
            
            # PIL을 사용하여 이미지 형식 확인 (imghdr 대신)
            image_format = check_image_format(file_path)
            
            if image_format in ["jpeg", "jpg", "png", "gif"]: 
                # 이미지 파일 열기
                img = Image.open(file_path)
                
                # RGB 모드로 변환 (RGBA나 다른 모드 처리)
                if img.mode != 'RGB':
                    img = img.convert('RGB')
                
                # 이미지 크기를 80x80으로 통일 (모델 입력 크기 일관성을 위함)
                resize_img = img.resize((80, 80))
                
                # PIL Image를 NumPy 배열로 변환
                pixel = np.array(resize_img)
                
                # RGB 3채널 이미지인지 확인 (80x80x3)
                if pixel.shape == (80, 80, 3):
                    data.append(pixel)
                    labels.append(label)
                    file_count += 1
                    
                    # 진행 상황 출력 (50개마다)
                    if file_count % 50 == 0:
                        print(f"  ✅ {file_count}개 파일 처리 완료...")
                        
                img.close()  # 메모리 해제
                    
        except Exception as e:
            error_count += 1
            if error_count <= 5:  # 처음 5개 오류만 출력
                print(f"  ❌ {filename} 처리 중 오류 발생: {e}")
            elif error_count == 6:
                print(f"  ⚠️ 너무 많은 오류로 출력을 제한합니다...")

    # 저장할 파일명 결정 (train 또는 test)
    title = "train" if isTrain else "test"
         
    # NPZ 파일로 저장
    npz_dir = "C:/Users/ryan9/문서/GitHub/SeSac-AI-Developer-Notes-2025/07_Deep_Learning/data/npz/"
    
    # npz 디렉토리가 없으면 생성
    os.makedirs(npz_dir, exist_ok=True)
    
    savefileName = npz_dir + f"imagedata{label}_{title}.npz"
    
    # 데이터와 레이블을 NPZ 형식으로 압축 저장
    np.savez(savefileName, data=data, targets=labels)
    
    print(f"💾 {title} 데이터 저장 완료!")
    print(f"   - 파일명: {savefileName}")
    print(f"   - 처리된 이미지 수: {len(data)}개")
    print(f"   - 오류 발생: {error_count}개")
    if len(data) > 0:
        print(f"   - 데이터 shape: {np.array(data).shape}")
    
    return len(data)

print("🎯 makeData 함수 정의 완료! (imghdr 없이 PIL 사용)")


🎯 makeData 함수 정의 완료! (imghdr 없이 PIL 사용)


In [8]:
# 데이터 생성 실행
print("🌸 꽃 분류 데이터 생성을 시작합니다!")
print("=" * 50)

# 데이지 학습 데이터 생성 (레이블 0)
print("🌼 데이지 학습 데이터 처리 중...")
daisy_train_count = makeData("daisy", 0, True)

print("\n" + "=" * 50)
print(f"✅ 데이지 학습 데이터 생성 완료! ({daisy_train_count}개 이미지)")


🌸 꽃 분류 데이터 생성을 시작합니다!
🌼 데이지 학습 데이터 처리 중...
📂 처리할 경로: C:/Users/ryan9/문서/GitHub/SeSac-AI-Developer-Notes-2025/07_Deep_Learning/data/flowers/train/daisy
  ✅ 50개 파일 처리 완료...
  ✅ 100개 파일 처리 완료...
  ✅ 150개 파일 처리 완료...
  ✅ 200개 파일 처리 완료...
  ✅ 250개 파일 처리 완료...
  ✅ 300개 파일 처리 완료...
  ✅ 350개 파일 처리 완료...
  ✅ 400개 파일 처리 완료...
  ✅ 450개 파일 처리 완료...
  ✅ 500개 파일 처리 완료...
💾 train 데이터 저장 완료!
   - 파일명: C:/Users/ryan9/문서/GitHub/SeSac-AI-Developer-Notes-2025/07_Deep_Learning/data/npz/imagedata0_train.npz
   - 처리된 이미지 수: 529개
   - 오류 발생: 0개
   - 데이터 shape: (529, 80, 80, 3)

✅ 데이지 학습 데이터 생성 완료! (529개 이미지)


In [9]:
# 추가 데이터 생성 옵션들 (필요시 주석 해제하여 실행)

print("📝 다음 코드들의 주석을 해제하여 추가 데이터를 생성할 수 있습니다:")
print("💡 레이블: 0=데이지(daisy), 1=민들레(dandelion)")
print("\n" + "-" * 50)

# 데이지 테스트 데이터 생성
# print("🌼 데이지 테스트 데이터 처리 중...")
# daisy_test_count = makeData("daisy", 0, False)
# print(f"✅ 데이지 테스트 데이터 생성 완료! ({daisy_test_count}개 이미지)\n")

# 민들레 학습 데이터 생성  
# print("🌻 민들레 학습 데이터 처리 중...")
# dandelion_train_count = makeData("dandelion", 1, True)
# print(f"✅ 민들레 학습 데이터 생성 완료! ({dandelion_train_count}개 이미지)\n")

# 민들레 테스트 데이터 생성
# print("🌻 민들레 테스트 데이터 처리 중...")
# dandelion_test_count = makeData("dandelion", 1, False)  
# print(f"✅ 민들레 테스트 데이터 생성 완료! ({dandelion_test_count}개 이미지)\n")

print("⚡ 한 번에 모든 데이터를 생성하려면 위의 모든 주석을 해제하세요!")


📝 다음 코드들의 주석을 해제하여 추가 데이터를 생성할 수 있습니다:
💡 레이블: 0=데이지(daisy), 1=민들레(dandelion)

--------------------------------------------------
⚡ 한 번에 모든 데이터를 생성하려면 위의 모든 주석을 해제하세요!


In [10]:
# 생성된 데이터 확인
npz_path = "C:/Users/ryan9/문서/GitHub/SeSac-AI-Developer-Notes-2025/07_Deep_Learning/data/npz/"

print("📋 생성된 NPZ 파일 목록:")
print("-" * 40)

if os.path.exists(npz_path):
    npz_files = [f for f in os.listdir(npz_path) if f.endswith('.npz')]
    
    if npz_files:
        total_images = 0
        for i, filename in enumerate(npz_files, 1):
            file_path = os.path.join(npz_path, filename)
            file_size = os.path.getsize(file_path) / (1024 * 1024)  # MB 단위
            print(f"{i}. {filename} ({file_size:.2f} MB)")
            
            # NPZ 파일 내용 확인
            try:
                data = np.load(file_path)
                data_shape = np.array(data['data']).shape
                labels_set = set(data['targets'])
                image_count = data_shape[0]
                total_images += image_count
                
                print(f"   📊 데이터 shape: {data_shape}")
                print(f"   🏷️  레이블 정보: {labels_set}")
                print(f"   🖼️  이미지 개수: {image_count}개")
                print()
                
            except Exception as e:
                print(f"   ❌ 파일 읽기 오류: {e}")
                print()
        
        print("=" * 40)
        print(f"🎯 총 처리된 이미지 수: {total_images}개")
        print("✅ 데이터 전처리가 성공적으로 완료되었습니다!")
        
    else:
        print("❌ NPZ 파일이 없습니다. 데이터를 먼저 생성해주세요.")
else:
    print("❌ NPZ 디렉토리가 존재하지 않습니다.")

print("\n🎉 꽃 분류 데이터 전처리 노트북 완료!")


📋 생성된 NPZ 파일 목록:
----------------------------------------
1. imagedata0_train.npz (9.69 MB)
   📊 데이터 shape: (529, 80, 80, 3)
   🏷️  레이블 정보: {np.int64(0)}
   🖼️  이미지 개수: 529개

🎯 총 처리된 이미지 수: 529개
✅ 데이터 전처리가 성공적으로 완료되었습니다!

🎉 꽃 분류 데이터 전처리 노트북 완료!
